In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
#import random as rn
#import re
import shutil

In [2]:
CSV_COLUMNS = ['Survived', 'Age', 'SibSp','Parch', 'C','Q', 'S','female', 'male', 'Pclass_1', 'Pclass_2', 'Pclass_3']
FEATURES = CSV_COLUMNS[1:len(CSV_COLUMNS)]     # our feature columns
LABEL = CSV_COLUMNS[0]                          #label columns
df_train = pd.read_csv('train.csv')           # training data
df_test = pd.read_csv('test.csv')             # test data
df1 = pd.read_csv('gender_submission.csv')               # now creating a evaluation data
df_eval = pd.merge(df1,df_test,on = 'PassengerId')
df_eval.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
def final_features(df):
    df.drop('Name', axis=1,inplace=True)                             # drop some useless columns
    df.drop('Fare', axis=1, inplace=True)
    df.drop('Ticket', axis=1, inplace=True)
    df.drop('PassengerId', axis=1, inplace=True)
    df.Embarked = df['Embarked'].fillna('S') # filling missing values,this column was missing only two values
    dummies = pd.get_dummies(df['Embarked']) #  creating dommies for categorical columns
    df = pd.concat([df,dummies],axis=1)
    dummies1 = pd.get_dummies(df['Sex'])
    df = pd.concat([df,dummies1],axis=1)
    df.drop('Sex', axis=1, inplace=True)      # dropping original columns as we have dommies
    df.drop('Embarked', axis=1, inplace=True)
    df.drop('Cabin', axis=1, inplace=True)
   # cbn = [ 'C23', 'C25', 'C27']
    #df.Cabin = df['Cabin'].fillna(rn.choice(cbn))
    df.Age = df['Age'].fillna(df.Age.median())   # filling missing values with median
    df.Age = (df['Age'] - df['Age'].mean()) / (df['Age'].max() - df['Age'].min())  # normalising the age column

    df = df.dropna()
    return df



In [4]:
# this is our train input function

def make_train_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = final_features(df),
    y = df[LABEL],
    #batch_size = 128,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 1000
    
  )

In [5]:
# def get_categorical1(name, values):
#     return tf.feature_column.indicator_column(categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(key = name, vocabulary_list = values))
# def get_categorical2(name2, values2):
#     return tf.feature_column.indicator_column(categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(key = name2, vocabulary_list = values2))
            
# this are our feature columns            

def make_feature_cols():
  return [   
    tf.feature_column.numeric_column('Pclass'),
    tf.feature_column.numeric_column('Age'),
    tf.feature_column.numeric_column('SibSp'),
    tf.feature_column.numeric_column('Parch'),
    tf.feature_column.numeric_column('C'),
    tf.feature_column.numeric_column('Q'),
    tf.feature_column.numeric_column('S'),
    tf.feature_column.numeric_column('female'),
    tf.feature_column.numeric_column('male')
  ]

In [6]:
tf.logging.set_verbosity(tf.logging.INFO)
OUTDIR = 'trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time
model = tf.estimator.DNNClassifier(hidden_units = [64,8, 8],feature_columns = make_feature_cols(),optimizer=tf.train.ProximalAdagradOptimizer(learning_rate=0.1,l1_regularization_strength=0.001), model_dir = OUTDIR)
model.train(input_fn = make_train_input_fn(df_train, num_epochs = 2000))


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'trained', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb9d6dca128>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
To construct input pipelines, use the `tf.data` mod

INFO:tensorflow:loss = 40.382248, step = 5501 (0.222 sec)
INFO:tensorflow:global_step/sec: 443.027
INFO:tensorflow:loss = 45.2237, step = 5601 (0.227 sec)
INFO:tensorflow:global_step/sec: 418.508
INFO:tensorflow:loss = 38.274868, step = 5701 (0.238 sec)
INFO:tensorflow:global_step/sec: 450.954
INFO:tensorflow:loss = 28.854958, step = 5801 (0.221 sec)
INFO:tensorflow:global_step/sec: 452.944
INFO:tensorflow:loss = 41.998314, step = 5901 (0.221 sec)
INFO:tensorflow:global_step/sec: 454.938
INFO:tensorflow:loss = 38.44822, step = 6001 (0.219 sec)
INFO:tensorflow:global_step/sec: 444.313
INFO:tensorflow:loss = 36.173008, step = 6101 (0.225 sec)
INFO:tensorflow:global_step/sec: 462.708
INFO:tensorflow:loss = 41.652206, step = 6201 (0.217 sec)
INFO:tensorflow:global_step/sec: 447.248
INFO:tensorflow:loss = 37.04979, step = 6301 (0.223 sec)
INFO:tensorflow:global_step/sec: 422.961
INFO:tensorflow:loss = 40.25541, step = 6401 (0.237 sec)
INFO:tensorflow:global_step/sec: 433.168
INFO:tensorflow

INFO:tensorflow:loss = 44.64332, step = 13801 (0.223 sec)
INFO:tensorflow:global_step/sec: 433.348
INFO:tensorflow:loss = 34.668823, step = 13901 (0.231 sec)
INFO:tensorflow:Saving checkpoints for 13922 into trained/model.ckpt.
INFO:tensorflow:Loss for final step: 22.108849.


In [7]:
# this is our evaluation input function


def make_eval_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = final_features(df),
    y = df[LABEL],
    batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
  )

In [8]:
#def print_eval(model, df):
  
result = model.evaluate(input_fn = make_eval_input_fn(df_eval))
print("The accuracy of our model is {} % ".format(100 * result['accuracy']) )
#print_eval(model, df_eval)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-17T13:20:33Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from trained/model.ckpt-13922
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-17-13:20:33
INFO:tensorflow:Saving dict for global step 13922: accuracy = 0.8492823, accuracy_baseline = 0.6363636, auc = 0.8633384, auc_precision_recall = 0.843297, average_loss = 0.7907363, global_step = 13922, label/mean = 0.36363637, loss = 82.63194, precision = 0.81560284, prediction/mean = 0.39015666, recall = 0.7565789
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 13922: trained/model.ckpt-13922
The accuracy of our model is 84.92823243141

In [9]:
# this is our prediction input function


def make_prediction_input_fn(df):
  return tf.estimator.inputs.pandas_input_fn(
    x = final_features(df),
    y = None,
   # batch_size = 128,
    shuffle = False,
    queue_capacity = 1000
    
  )


In [10]:
# these are predictions
#data = pd.DataFrame(columns=['PassengerId', 'Survived'])
#data.index = np.arange(892, len(data) + 1)
#data['PassengerId'] = data.index
predictions = model.predict(input_fn = make_prediction_input_fn(df_test))
p_id = np.empty((0))
survived = np.empty((0))
i = 892
for items in predictions:
        result = items['probabilities'].argmax(axis=0)
        survived = np.append(survived, [result], axis=0)
        p_id = np.append(p_id, [i], axis=0)
        i = i + 1
        

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from trained/model.ckpt-13922
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [11]:
data = pd.DataFrame({'PassengerId':p_id,'Survived':survived})
data['PassengerId'] = data['PassengerId'].astype('int')
data['Survived'] = data['Survived'].astype('int')
data.to_csv('submission.csv')
print(data.describe())


       PassengerId    Survived
count   418.000000  418.000000
mean   1100.500000    0.337321
std     120.810458    0.473362
min     892.000000    0.000000
25%     996.250000    0.000000
50%    1100.500000    0.000000
75%    1204.750000    1.000000
max    1309.000000    1.000000
